In [2]:
# Prerequisites
import numpy as np

def hammingdistance(seq1, seq2):
    seq1 = list(seq1)
    seq2 = list(seq2)
    hmgdist = 0
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            hmgdist += 1
    return hmgdist

def Suffix(Pattern):
    Pattern = list(Pattern)
    Pattern.pop(0)
    return "".join(Pattern)

def Neighbors(Pattern, d):
    Nucleotides = ["A", "C", "T", "G"]
    if d == 0:
        return Pattern
    if len(Pattern) == 1:
        return Nucleotides
    
    Neighborhood = []
    SuffixNeighbors = Neighbors(Suffix(Pattern), d)
    for i in SuffixNeighbors:
        if hammingdistance(Suffix(Pattern), i) < d:
            for nt in Nucleotides:
                Neighborhood.append(nt + i)
        else:
            PatternList = list(Pattern)
            Neighborhood.append(PatternList[0] + i)
    
    return Neighborhood

def ReverseComplement(Pattern):
    Pattern = Pattern[::-1]
    Pattern_list = list(Pattern)
    for i in range(0, len(Pattern)):
        if Pattern_list[i] == "A":
            Pattern_list[i] = "T"
        elif Pattern_list[i] == "T":
            Pattern_list[i] = "A"
        elif Pattern_list[i] == "C":
            Pattern_list[i] = "G"
        elif Pattern_list[i] == "G":
            Pattern_list[i] = "C"
    result = ""
    for i in Pattern_list:
        result += i
    return result

def minHamDist(pattern1, pattern2):
    # Pattern1 should be shorter than pattern2
    if len(pattern1) > len(pattern2):
        pattern1, pattern2 = pattern2, pattern1
    len1 = len(pattern1)
    len2 = len(pattern2)
    minDist = len(pattern1)

    for i in range(0, len2-len1+1):
        kmer = pattern2[i:i+len1]
        if hammingdistance(pattern1, kmer) < minDist:
            minDist = hammingdistance(pattern1, kmer)
    
    return minDist

In [3]:
# (k, d)-motif: a k-mer if it appears in every string from Dna with at most d mismatches
# Brute force, generating all possible k-mers

def MotifEnumeration(Dna, k, d):
    Patterns = []
    Dna = Dna.split()

    line1 = Dna[0]
    kmerlist_line1 = []
    for i in range(0, len(line1)-k+1):
        kmer = line1[i:i+k]
        kmerlist_line1.append(kmer)

   # Creating a list of all k-mer neighbor differing from Pattern by at most d mismatches 
    all_neighbors = []
    all_neighbors += kmerlist_line1
    for kmer in kmerlist_line1:
        neighbors = Neighbors(kmer, d)
        all_neighbors += neighbors
    all_neighbors = list(set(all_neighbors))

    # To make sure if there is one hit with maximum hamming distance of d on every single line of genomic DNA
    for neighbor in all_neighbors:
        criteria = 0
        for line in Dna[1:]:
            minDist = minHamDist(neighbor, line)
            if minDist <= d:
                criteria += 1
        if criteria == len(Dna[1:]):
            Patterns.append(neighbor)

    return list(set(Patterns))

In [4]:
import math

def entropy(seq):
    # seq input is a list with all capital sequences
    entropy = 0
    seqlength = len(seq[0])
    seqcount = len(seq)
    
    for i in range(0, seqlength):
        nucleotide_count = {"A":0, "T":0, "G":0, "C":0}
        for j in seq:
            nucleotide = j[i]
            nucleotide_count[nucleotide] += 1
        
        for nt in nucleotide_count:
            p_i = nucleotide_count[nt] / seqcount
            if p_i != 0:
                entropy -= p_i * math.log2(p_i)
            else:
                entropy += 0
    
    return entropy

In [5]:
def DistanceBetweenPatternAndStrings(Pattern, Dna):
    k = len(Pattern)
    Dna = Dna.split()
    distance = 0
    strlength = len(Dna[0])

    for str in Dna:
        kmerlist = []
        hammingdist = strlength
        for i in range(0, strlength-k+1):
            kmerlist.append(str[i:i+k])
        for kmer in kmerlist:
            if hammingdistance(Pattern, kmer) < hammingdist:
                hammingdist = hammingdistance(Pattern, kmer)
        distance += hammingdist
    
    return distance

In [6]:
def AllStrings(k):
    klist = []
    nucleotides = ["A", "T", "G", "C"]

    for nt in nucleotides:
        if k == 1:
            kmer = nt
            klist.append(kmer)
        else:
            for i in AllStrings(k-1):
                item = i + nt
                klist.append(item)
    
    return klist

In [16]:
def MedianString(Dna, k):
    Patterns = AllStrings(k)
    Median = min(Patterns, key = lambda Pattern: DistanceBetweenPatternAndStrings(Pattern, Dna.upper()))

    return Median
MedianString("ctcgatgagtaggaaagtagtttcactgggcgaaccaccccggcgctaatcctagtgccc gcaatcctacccgaggccacatatcagtaggaactagaaccaccacgggtggctagtttc ggtgttgaaccacggggttagtttcatctattgtaggaatcggcttcaaatcctacacag", 7)

'GTAGGAA'

In [8]:
def profile_most_probable(Text, k, prob):
    # The input matrix must be a string composed of numbers split by spaces only, e.g., "0.2 0.2 0.3 0.2 0.3 0.4 0.3 0.1 0.5 0.1 0.3 0.3 0.5 0.2 0.4 ... 0.1 0.2"
    list_prob = [x for x in prob.split()]
    list_prob = [list_prob[i:i+k] for i in range(0, len(list_prob), k)]
    matrix = {"A":list_prob[0], "C":list_prob[1], "G":list_prob[2], "T":list_prob[3]} # Stored in the form of a dict
        
    list_kmer = [Text[i:i+k] for i in range(0, len(Text)-k+1)]
    most_probable = ""
    max_prob = -99
    for kmer in list_kmer:
        prob = 1
        for i in range(0, k):
            nt = kmer[i]
            prob_nt = eval(matrix[nt][i])
            prob *= prob_nt
        if prob > max_prob:
            max_prob = prob
            most_probable = kmer
        
    return most_probable

# profile_most_probable("ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT", 5, "0.2 0.2 0.3 0.2 0.3 0.4 0.3 0.1 0.5 0.1 0.3 0.3 0.5 0.2 0.4 0.1 0.2 0.1 0.1 0.2")

In [9]:
def profile_string(motifs):
    if type(motifs) != list:
        motifs = motifs.split()
    nt_dict = {}
    for i in range(0, len(motifs[0])):
        for motif in motifs:
            if i not in nt_dict:
                nt_dict[i] = {"A":1, "C":1, "G":1, "T":1}
            nt_dict[i][motif[i]] += 1
    # nt_dict is a dict with a form of {i: {"A":count_A ..., "T":count_T}}, in which i is the position of nucleotide in a motif
    ntlist = ["A", "C", "G", "T"]
    probability = ""
    for nucleotide in ntlist:
        for i in range(0, len(nt_dict)):
            probability = probability + " " + str(nt_dict[i][nucleotide]/sum(nt_dict[i].values()))
    prob = probability[1:]
    return prob

profile_string("GGC")

'0.2 0.2 0.2 0.2 0.2 0.4 0.4 0.4 0.2 0.2 0.2 0.2'

In [10]:
def profile_dict(motifs):
    if type(motifs) != list:
        motifs = motifs.split()
    nt_dict = {}
    for i in range(0, len(motifs[0])):
        for motif in motifs:
            if i not in nt_dict:
                nt_dict[i] = {"A":1, "C":1, "G":1, "T":1}
            nt_dict[i][motif[i]] += 1
    # nt_dict is a dict with a form of {i: {"A":count_A ..., "T":count_T}}, in which i is the position of nucleotide in a motif
    return nt_dict
profile_dict("GGC")

{0: {'A': 1, 'C': 1, 'G': 2, 'T': 1},
 1: {'A': 1, 'C': 1, 'G': 2, 'T': 1},
 2: {'A': 1, 'C': 2, 'G': 1, 'T': 1}}

In [11]:
def score(motifs):
    nt_dict = profile_dict(motifs)
    final_score = 0
    for i in range(0, len(nt_dict)):
        max_count = max(nt_dict[i].values())
        final_score += sum(nt_dict[i].values()) - max_count
    return final_score

#score("TCGGGGGTTTTT CCGGTGACTTTC ACGGGGATTTTC TTGGGGACTTTT AAGGGGACTTCC TTGGGGACTTCC TCGGGGATTCAT TCGGGGATTCCT TAGGGGAACTAC TCGGGTATAACC")

In [15]:
def GreedyMotifSearch(Dna, k, t):
    Dna = Dna.split()
    BestMotifs = [line[:k] for line in Dna]

    Motifs_line1 = [Dna[0][i:i+k] for i in range(0, len(Dna[0])-k+1)]
    for kmer in Motifs_line1:
        Motifs = [kmer]
        for i in range(1, t):
            Motifs.append(profile_most_probable(Dna[i], k, profile_string(Motifs)))
        if score(Motifs) < score(BestMotifs):
            BestMotifs = Motifs
    for i in BestMotifs:
        print(i, end=" ")
    return BestMotifs